In [ ]:
import seaborn as sns
sns.set_style('whitegrid')
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import os
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

> Reading the new CSV file which was created after appending all the CSV files

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filename

> Reading the file

In [ ]:
df=pd.read_csv('/kaggle/input/datafinal/datafinal.csv')

In [ ]:
df

In [ ]:
df.info()


In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
drop_cols = ['Unnamed: 0']
df = df.drop(drop_cols, axis=1)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.shape

> Checking if there are any missing values

In [ ]:
df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# columns to select for encoding
selected_col = ['Time']

le.fit(df[selected_col].values.flatten())

df[selected_col] = df[selected_col].apply(le.fit_transform)

> Copying the Dataframe to **a** which is used in the model

In [ ]:
a=df.copy()

In [ ]:
df

In [ ]:
a

# **DATA PLOTTING**

In [ ]:
#Plotting data 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [ ]:

#All features distribution
i=1
plt.figure(figsize=(25,20))
for c in df.describe().columns[:]:
    plt.subplot(5,3,i)
    plt.title(f"Histogram of {c}",fontsize=10)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    plt.hist(df[c],bins=20,color='teal',edgecolor='k')
    i+=1
plt.show()

In [ ]:
#Box Plotting All features distribution corresponding Target column
i=1
plt.figure(figsize=(40,40))
for c in df.columns[:14]:
    plt.subplot(3,5,i)
    plt.title(f"Boxplot of {c}",fontsize=16)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    sns.boxplot(y=df[c],x=df['Pos'])
    i+=1
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,4))

ax=fig.add_subplot(121)
sns.distplot(df['Pos'],bins=50,color='r',ax=ax)
ax.set_title('Distribution of data pos')

ax=fig.add_subplot(122)
sns.distplot(np.log10(df['Pos']),bins=40,color='b',ax=ax)
ax.set_title('Distribution of data Poses in $log$ sacle')
ax.set_xscale('log')
""

# **Feature Diagram**

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# machine learning model

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
a

In [ ]:
y= a['Pos']

X = a.drop(['Pos'],axis = 1)

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
rfc =RandomForestClassifier(n_estimators=100)
rfc.fit(X,y)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [ ]:
imp_features = rfc.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfc.estimators_], axis = 0)
 
plt.figure(figsize = (15,8))
plt.bar(X.columns, std, color = 'teal') 
plt.xlabel('Feature Labels') 
plt.ylabel('Feature Importances') 
plt.title('Comparison of different Feature Importances') 
plt.show()

# I will need to do data cleaning if the result is overfitting

* UPDATE: OVERFITTING
* UPDATE: After setting 0.01 as min and 0.04 as max, overfitting is solved

In [ ]:
#Data Cleaning
drop_cols = ['Time','UNIX_T','Lux','Temp','Co','Ro']
a = a.drop(drop_cols, axis=1)

In [ ]:
a

In [ ]:
a.info()

>Feature Extraction, Importance & Splitting

In [ ]:
y= a['Pos']

X = a.drop(['Pos'],axis = 1)

In [ ]:
y

In [ ]:
X

>Splitting the dataset


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2021)

In [ ]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)


In [ ]:
print("Shape of y_train: ",y_train.shape)
print("Shape of y_test",y_test.shape)

> Data Standardization / Feature Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:

# Feature Scaling
sc =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

> Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
training_start = time.perf_counter()
rfc.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = rfc.predict(X_test)
prediction_end = time.perf_counter()
acc_rfc = (preds == y_test).sum().astype(float) / len(preds)*100
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_rfc))
print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

In [ ]:
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_rfc))

# Cross Validation

In [ ]:
rfc_cv = RandomForestClassifier(max_depth=10, n_estimators=100)

scores = cross_val_score(rfc_cv, X_train, y_train, cv=5, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


In [ ]:

from sklearn.model_selection import train_test_split, cross_val_predict
predictions = cross_val_predict(rfc_cv, df.drop('Pos', axis=1), df['Pos'], cv=5)
confusion_matrix(df['Pos'], predictions)

In [ ]:

print("Precision:", precision_score(df['Pos'], predictions, average='micro'))

print("Recall:",recall_score(df['Pos'], predictions, average='micro'))

In [ ]:


print("F1-Score:", f1_score(df['Pos'], predictions, average='micro'))
